# Assignment 4: Self-Attention for Vision

For this assignment, we're going to implement self-attention blocks in a convolutional neural network for CIFAR-10 Classification.

# Part I. Preparation

First, we load the CIFAR-10 dataset. This might take a couple minutes the first time you do it, but the files should stay cached after that.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

In [2]:
NUM_TRAIN = 49000

# The torchvision.transforms package provides tools for preprocessing data
# and for performing data augmentation; here we set up a transform to
# preprocess the data by subtracting the mean RGB value and dividing by the
# standard deviation of each RGB value; we've hardcoded the mean and std.
transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])

# We set up a Dataset object for each split (train / val / test); Datasets load
# training examples one at a time, so we wrap each Dataset in a DataLoader which
# iterates through the Dataset and forms minibatches. We divide the CIFAR-10
# training set into train and val sets by passing a Sampler object to the
# DataLoader telling how it should sample from the underlying Dataset.
cifar10_train = dset.CIFAR10('./data/datasets', train=True, download=True,
                             transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./data/datasets', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./data/datasets', train=False, download=True, 
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


You have an option to **use GPU by setting the flag to True below**. It is not necessary to use GPU for this assignment. Note that if your computer does not have CUDA enabled, `torch.cuda.is_available()` will return False and this notebook will fallback to CPU mode.

The global variables `dtype` and `device` will control the data types throughout this assignment. 

In [3]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cuda


## Flatten Function

In [4]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

def test_flatten():
    x = torch.arange(12).view(2, 1, 3, 2)
    print('Before flattening: ', x)
    print('After flattening: ', flatten(x))

test_flatten()

Before flattening:  tensor([[[[ 0,  1],
          [ 2,  3],
          [ 4,  5]]],


        [[[ 6,  7],
          [ 8,  9],
          [10, 11]]]])
After flattening:  tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11]])


### Check Accuracy Function


In [5]:
import torch.nn.functional as F  # useful stateless functions
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
        return 100 * acc

### Training Loop

In [6]:
def train(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    acc_max = 0
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Epoch %d, Iteration %d, loss = %.4f' % (e, t, loss.item()))
                acc = check_accuracy(loader_val, model)
                if acc >= acc_max:
                    acc_max = acc
                print()
    print("Maximum accuracy attained: ", acc_max)

In [7]:
# We need to wrap `flatten` function in a module in order to stack it
# in nn.Sequential
class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

## Vanilla CNN; No Attention
We implement the vanilla architecture for you here. Do not modify the architecture. You will use the same architecture in the following parts. Do not modify the hyper-parameters.

In [12]:
channel_1 = 64
channel_2 = 32
learning_rate = 1e-3
num_classes = 10

model = nn.Sequential(
    nn.Conv2d(3, channel_1, 3, padding=1, stride=1),
    nn.ReLU(),
    nn.Conv2d(channel_1, channel_2, 3, padding=1),
    nn.ReLU(),
    Flatten(),
    nn.Linear(channel_2*32*32, num_classes),
)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)


train(model, optimizer, epochs=10)

Epoch 0, Iteration 0, loss = 2.3160
Checking accuracy on validation set
Got 147 / 1000 correct (14.70)

Epoch 0, Iteration 100, loss = 1.7426
Checking accuracy on validation set
Got 411 / 1000 correct (41.10)

Epoch 0, Iteration 200, loss = 1.6827
Checking accuracy on validation set
Got 447 / 1000 correct (44.70)

Epoch 0, Iteration 300, loss = 1.2107
Checking accuracy on validation set
Got 489 / 1000 correct (48.90)

Epoch 0, Iteration 400, loss = 1.1421
Checking accuracy on validation set
Got 530 / 1000 correct (53.00)

Epoch 0, Iteration 500, loss = 1.3717
Checking accuracy on validation set
Got 549 / 1000 correct (54.90)

Epoch 0, Iteration 600, loss = 1.2229
Checking accuracy on validation set
Got 556 / 1000 correct (55.60)

Epoch 0, Iteration 700, loss = 1.2221
Checking accuracy on validation set
Got 564 / 1000 correct (56.40)

Epoch 1, Iteration 0, loss = 0.9969
Checking accuracy on validation set
Got 573 / 1000 correct (57.30)

Epoch 1, Iteration 100, loss = 1.1605
Checking acc

Epoch 9, Iteration 600, loss = 0.4219
Checking accuracy on validation set
Got 590 / 1000 correct (59.00)

Epoch 9, Iteration 700, loss = 0.7132
Checking accuracy on validation set
Got 590 / 1000 correct (59.00)

Maximum accuracy attained:  64.1


## Test set -- run this only once

Now we test our model on the test set . Think about how this compares to your validation set accuracy.
You should be able to see atleast 55% accuracy

In [13]:
vanillaModel = model
check_accuracy(loader_test, vanillaModel)


Checking accuracy on test set
Got 5798 / 10000 correct (57.98)


57.98

## Part II Self-Attention

In the next section, you will implement an Attention layer which you will then use within a convnet architecture defined above for cifar 10 classification task.

A self-attention layer is formulated as following:

Input: $X$ of shape $(H\times W, C)$

Query, key, value linear transforms are $W_Q$, $W_K$, $W_V$, of shape $(C, C)$. We implement these linear transforms as 1x1 convolutional layers of the same dimensions.

$XW_Q$, $XW_K$, $XW_V$, represent the output volumes when input X is passed through the transforms.


Self-Attention is given by the formula: $Attention(X) = X + Softmax(\frac{XW_Q(XW_K)^\top}{\sqrt{C}})XW_V$

### Inline Question 1: Self-Attention is equivalent to which of the following: (5 points)
1. K-means clustering <br />
2. Non-local means <br />
3. Residual Block <br />
4. Gaussian Blurring <br />

Your Answer: Self-attention is equivalent to non-local means. The non-local means algorithm replaces the value of a pixel by an average of a selection of other pixels values, while self-attention is an attention mechanism relating different positions of a single sequence in order to compute a representation of the same sequence. K-means is a method of vector quantization that aims to partition n observations into k clusters in which each observation belongs to the cluster with the nearest mean. A residual block is a stack of layers set in such a way that the output of a layer is taken and added to another layer deeper in the block, but it includes no attention. Guassian blurring is to blur an image by Gaussian function. 

### Here you implement the Attention module, and run it in the next section (40 points)

In [60]:
# Initialize the attention module as a nn.Module subclass
class Attention(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        
        # TODO: Implement the Key, Query and Value linear transforms as 1x1 convolutional layers
        # Hint: channel size remains constant throughout
        self.conv_query = nn.Conv2d(in_channels, in_channels, 1)
        self.conv_key = nn.Conv2d(in_channels, in_channels, 1)
        self.conv_value = nn.Conv2d(in_channels, in_channels, 1)

    def forward(self, x):
        N, C, H, W = x.shape
        
        # TODO: Pass the input through conv_query, reshape the output volume to (N, C, H*W)
        q = self.conv_query(x).reshape(N, C, H*W)
        # TODO: Pass the input through conv_key, reshape the output volume to (N, C, H*W)
        k = self.conv_key(x).reshape(N, C, H*W)
        # TODO: Pass the input through conv_value, reshape the output volume to (N, C, H*W)
        v = self.conv_value(x).reshape(N, C, H*W)
        # TODO: Implement the above formula for attention using q, k, v, C
        # NOTE: The X in the formula is already added for you in the return line
        temp = torch.matmul(q, torch.transpose(k, 1, 2))/(np.sqrt(C))
        attention = torch.matmul(F.softmax(temp, dim=-1), v)
        # Reshape the output to (N, C, H, W) before adding to the input volume
        attention = attention.reshape(N, C, H, W)
        return x + attention

## Single Attention Block: Early attention; After the first conv layer. (10 points)

In [77]:
channel_1 = 64
channel_2 = 32
learning_rate = 1e-3

# TODO: Use the above Attention module after the first Convolutional layer.
# Essentially the architecture should be [Conv->Relu->Attention->Relu->Conv->Relu->Linear]

model = nn.Sequential(
    nn.Conv2d(3, channel_1, 3, padding=1, stride=1),
    nn.ReLU(),
    Attention(channel_1),
    nn.ReLU(),
    nn.Conv2d(channel_1, channel_2, 3, padding=1),
    nn.ReLU(),
    Flatten(),
    nn.Linear(channel_2*32*32, 10),
)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, optimizer, epochs=10)

Epoch 0, Iteration 0, loss = 2.3018
Checking accuracy on validation set
Got 135 / 1000 correct (13.50)

Epoch 0, Iteration 100, loss = 1.4160
Checking accuracy on validation set
Got 385 / 1000 correct (38.50)

Epoch 0, Iteration 200, loss = 1.4014
Checking accuracy on validation set
Got 468 / 1000 correct (46.80)

Epoch 0, Iteration 300, loss = 1.4034
Checking accuracy on validation set
Got 533 / 1000 correct (53.30)

Epoch 0, Iteration 400, loss = 1.5503
Checking accuracy on validation set
Got 555 / 1000 correct (55.50)

Epoch 0, Iteration 500, loss = 1.1840
Checking accuracy on validation set
Got 564 / 1000 correct (56.40)

Epoch 0, Iteration 600, loss = 1.1146
Checking accuracy on validation set
Got 581 / 1000 correct (58.10)

Epoch 0, Iteration 700, loss = 1.0500
Checking accuracy on validation set
Got 612 / 1000 correct (61.20)

Epoch 1, Iteration 0, loss = 1.1237
Checking accuracy on validation set
Got 624 / 1000 correct (62.40)

Epoch 1, Iteration 100, loss = 1.0338
Checking acc

Epoch 9, Iteration 600, loss = 0.2202
Checking accuracy on validation set
Got 632 / 1000 correct (63.20)

Epoch 9, Iteration 700, loss = 0.2224
Checking accuracy on validation set
Got 631 / 1000 correct (63.10)

Maximum accuracy attained:  67.5


## Test set -- run this only once

Now we test our model on the test set . Think about how this compares to your validation set accuracy.
You should see improvement of about 2-3% over the vanilla convnet model. * Use this part to tune your Attention module and then move on to the next parts. *

In [78]:
earlyAttention = model
check_accuracy(loader_test, earlyAttention)

Checking accuracy on test set
Got 6143 / 10000 correct (61.43)


61.42999999999999

## Single Attention Block: Late attention; After the second conv layer. (10 points)

In [79]:
channel_1 = 64
channel_2 = 32
learning_rate = 1e-3

# TODO: Use the above Attention module after the Second Convolutional layer.
# Essentially the architecture should be [Conv->Relu->Conv->Relu->Attention->Relu->Linear]

model = nn.Sequential(
    nn.Conv2d(3, channel_1, 3, padding=1, stride=1),
    nn.ReLU(),
    nn.Conv2d(channel_1, channel_2, 3, padding=1),
    nn.ReLU(),
    Attention(channel_2),
    nn.ReLU(),
    Flatten(),
    nn.Linear(channel_2*32*32, 10),
)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, optimizer, epochs=10)

Epoch 0, Iteration 0, loss = 2.3077
Checking accuracy on validation set
Got 168 / 1000 correct (16.80)

Epoch 0, Iteration 100, loss = 1.4646
Checking accuracy on validation set
Got 419 / 1000 correct (41.90)

Epoch 0, Iteration 200, loss = 1.2998
Checking accuracy on validation set
Got 476 / 1000 correct (47.60)

Epoch 0, Iteration 300, loss = 1.2162
Checking accuracy on validation set
Got 530 / 1000 correct (53.00)

Epoch 0, Iteration 400, loss = 1.4571
Checking accuracy on validation set
Got 503 / 1000 correct (50.30)

Epoch 0, Iteration 500, loss = 1.1879
Checking accuracy on validation set
Got 559 / 1000 correct (55.90)

Epoch 0, Iteration 600, loss = 1.1361
Checking accuracy on validation set
Got 581 / 1000 correct (58.10)

Epoch 0, Iteration 700, loss = 1.4091
Checking accuracy on validation set
Got 565 / 1000 correct (56.50)

Epoch 1, Iteration 0, loss = 1.0744
Checking accuracy on validation set
Got 597 / 1000 correct (59.70)

Epoch 1, Iteration 100, loss = 1.1168
Checking acc

Epoch 9, Iteration 600, loss = 0.3285
Checking accuracy on validation set
Got 611 / 1000 correct (61.10)

Epoch 9, Iteration 700, loss = 0.1595
Checking accuracy on validation set
Got 612 / 1000 correct (61.20)

Maximum accuracy attained:  65.2


## Test set -- run this only once

Now we test our model on the test set . Think about how this compares to your validation set accuracy.

In [80]:
lateAttention = model
check_accuracy(loader_test, lateAttention)

Checking accuracy on test set
Got 6174 / 10000 correct (61.74)


61.739999999999995

### Inline Question 2: Provide one example each of usage of self-attention and attention in computer vision. Explain the difference between the two. (5 points)


Your Answer: The main difference between the self-attention and attention mechanism is that self-attention can only learn attention from its own layer while the attention mechanism can learn it from other layers.

Attention Mechanism: Convolutional Block Attention Module

Self-attention Mechanism: Self-Attention Generative Adversarial Networks

## Double Attention Blocks: After conv layers 1 and 2 (10 points)

In [65]:
channel_1 = 64
channel_2 = 32
learning_rate = 1e-3

# TODO: Use the above Attention module after the Second Convolutional layer.
# Essentially the architecture should be [Conv->Relu->Attention->Relu->Conv->Relu->Attention->Relu->Linear]

model = nn.Sequential(
    nn.Conv2d(3, channel_1, 3, padding=1, stride=1),
    nn.ReLU(),
    Attention(channel_1),
    nn.ReLU(),
    nn.Conv2d(channel_1, channel_2, 3, padding=1),
    nn.ReLU(),
    Attention(channel_2),
    nn.ReLU(),
    Flatten(),
    nn.Linear(channel_2*32*32, 10),
)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, optimizer, epochs=10)

Epoch 0, Iteration 0, loss = 2.3040
Checking accuracy on validation set
Got 119 / 1000 correct (11.90)

Epoch 0, Iteration 100, loss = 1.5456
Checking accuracy on validation set
Got 429 / 1000 correct (42.90)

Epoch 0, Iteration 200, loss = 1.4643
Checking accuracy on validation set
Got 484 / 1000 correct (48.40)

Epoch 0, Iteration 300, loss = 1.5998
Checking accuracy on validation set
Got 515 / 1000 correct (51.50)

Epoch 0, Iteration 400, loss = 1.5197
Checking accuracy on validation set
Got 545 / 1000 correct (54.50)

Epoch 0, Iteration 500, loss = 1.0481
Checking accuracy on validation set
Got 552 / 1000 correct (55.20)

Epoch 0, Iteration 600, loss = 1.1773
Checking accuracy on validation set
Got 553 / 1000 correct (55.30)

Epoch 0, Iteration 700, loss = 0.9678
Checking accuracy on validation set
Got 566 / 1000 correct (56.60)

Epoch 1, Iteration 0, loss = 1.0036
Checking accuracy on validation set
Got 575 / 1000 correct (57.50)

Epoch 1, Iteration 100, loss = 1.1466
Checking acc

Epoch 9, Iteration 600, loss = 0.2645
Checking accuracy on validation set
Got 643 / 1000 correct (64.30)

Epoch 9, Iteration 700, loss = 0.1594
Checking accuracy on validation set
Got 639 / 1000 correct (63.90)

Maximum accuracy attained:  68.2


## Test set -- run this only once

Now we test our model on the test set . Think about how this compares to your validation set accuracy.

In [66]:
vanillaModel = model
check_accuracy(loader_test, vanillaModel)

Checking accuracy on test set
Got 6367 / 10000 correct (63.67)


63.67

## Resnet with Attention 

Now we will experiment with applying attention within the Resnet10 architecture that we implemented in Homework 2. Please note that for a deeper model such as Resnet we do not expect significant improvements in performance with Attention

## Vanilla Resnet, No Attention

The architecture for Resnet is given below, please train it and evaluate it on the test set.

In [67]:
import torch
import torch.nn as nn

class ResNet(nn.Module):

    def __init__(self, block, layers, img_channels=3, num_classes=100, batchnorm=False):
        super(ResNet, self).__init__() #layers = [1, 1, 1, 1] 
        self.in_channels = 64
        self.conv1 = nn.Conv2d(img_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.batchnorm = batchnorm
        self.layer1 = self.make_layer(block, layers[0], out_channels=64, stride=1, batchnorm=batchnorm)
        self.layer2 = self.make_layer(block, layers[1], out_channels=128, stride=1, batchnorm=batchnorm)
        self.layer3 = self.make_layer(block, layers[2], out_channels=256, stride=1, batchnorm=batchnorm)
        self.layer4 = self.make_layer(block, layers[3], out_channels=512, stride=2, batchnorm=batchnorm)

        self.averagepool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)
   
    def forward(self, x):

        x = self.conv1(x)
        if self.batchnorm:
            x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x) 
        x = self.layer2(x)       
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.averagepool(x)
        x = x.reshape(x.shape[0], -1)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)

        return x


        

    def make_layer(self, block, num_blocks, out_channels, stride, batchnorm=False):
        downsampler = None
        layers = []
        if stride != 1 or self.in_channels != out_channels:
            downsampler = nn.Sequential(nn.Conv2d(self.in_channels, out_channels, kernel_size = 1, stride = stride), nn.BatchNorm2d(out_channels))

        layers.append(block(self.in_channels, out_channels, downsampler, stride, batchnorm=batchnorm))

        self.in_channels = out_channels

        for i in range(num_blocks - 1):
            layers.append(block(self.in_channels, out_channels))

        
        return nn.Sequential(*layers)
        
class block(nn.Module):

    def __init__(self, in_channels, out_channels, downsampler = None, stride = 1, batchnorm=False):
        
        super(block, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 2)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = stride)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsampler = downsampler
        self.relu = nn.ReLU()
        self.batchnorm = batchnorm

    
    def forward(self, x):

        residual = x
        x = self.conv1(x)
        if self.batchnorm:
            x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        if self.batchnorm:
            x = self.bn2(x)
        x = self.relu(x)
        
        if self.downsampler:
            residual = self.downsampler(residual)

        return self.relu(residual + x)
    


def ResNet10(num_classes = 100, batchnorm= False):

    return ResNet(block, [1, 1, 1, 1], num_classes=num_classes, batchnorm=batchnorm)

In [68]:
learning_rate = 1e-3

model = ResNet10()

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, optimizer, epochs=10)

Epoch 0, Iteration 0, loss = 4.5434
Checking accuracy on validation set
Got 151 / 1000 correct (15.10)

Epoch 0, Iteration 100, loss = 1.4887
Checking accuracy on validation set
Got 388 / 1000 correct (38.80)

Epoch 0, Iteration 200, loss = 1.5201
Checking accuracy on validation set
Got 479 / 1000 correct (47.90)

Epoch 0, Iteration 300, loss = 1.2628
Checking accuracy on validation set
Got 486 / 1000 correct (48.60)

Epoch 0, Iteration 400, loss = 1.2302
Checking accuracy on validation set
Got 477 / 1000 correct (47.70)

Epoch 0, Iteration 500, loss = 0.9602
Checking accuracy on validation set
Got 530 / 1000 correct (53.00)

Epoch 0, Iteration 600, loss = 1.3069
Checking accuracy on validation set
Got 584 / 1000 correct (58.40)

Epoch 0, Iteration 700, loss = 1.2056
Checking accuracy on validation set
Got 582 / 1000 correct (58.20)

Epoch 1, Iteration 0, loss = 1.0182
Checking accuracy on validation set
Got 580 / 1000 correct (58.00)

Epoch 1, Iteration 100, loss = 1.0285
Checking acc

Epoch 9, Iteration 600, loss = 0.4068
Checking accuracy on validation set
Got 770 / 1000 correct (77.00)

Epoch 9, Iteration 700, loss = 0.2769
Checking accuracy on validation set
Got 773 / 1000 correct (77.30)

Maximum accuracy attained:  78.8


## Test set -- run this only once

Now we test our model on the test set . Think about how this compares to your validation set accuracy.

In [69]:
vanillaResnet = model
check_accuracy(loader_test, vanillaResnet)

Checking accuracy on test set
Got 7430 / 10000 correct (74.30)


74.3

## Resnet with Attention (5 points)

In [70]:
import torch
import torch.nn as nn

class ResNet_Attention(nn.Module):

    def __init__(self, block, layers, img_channels=3, num_classes=100, batchnorm=False):
        super(ResNet_Attention, self).__init__() #layers = [1, 1, 1, 1] 
        self.in_channels = 64
        self.conv1 = nn.Conv2d(img_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.batchnorm = batchnorm
        self.layer1 = self.make_layer(block, layers[0], out_channels=64, stride=1, batchnorm=batchnorm)
        self.layer2 = self.make_layer(block, layers[1], out_channels=128, stride=1, batchnorm=batchnorm)
        self.attention = Attention(128)
        self.layer3 = self.make_layer(block, layers[2], out_channels=256, stride=1, batchnorm=batchnorm)
        self.layer4 = self.make_layer(block, layers[3], out_channels=512, stride=2, batchnorm=batchnorm)

        self.averagepool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)
   
    def forward(self, x):

        x = self.conv1(x)
        if self.batchnorm:
            x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x) 
        x = self.layer2(x)
        x = self.attention(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.averagepool(x)
        x = x.reshape(x.shape[0], -1)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)

        return x

    def make_layer(self, block, num_blocks, out_channels, stride, batchnorm=False):
        downsampler = None
        layers = []
        if stride != 1 or self.in_channels != out_channels:
            downsampler = nn.Sequential(nn.Conv2d(self.in_channels, out_channels, kernel_size = 1, stride = stride), nn.BatchNorm2d(out_channels))

        layers.append(block(self.in_channels, out_channels, downsampler, stride, batchnorm=batchnorm))

        self.in_channels = out_channels

        for i in range(num_blocks - 1):
            layers.append(block(self.in_channels, out_channels))

        
        return nn.Sequential(*layers)
        
class block(nn.Module):

    def __init__(self, in_channels, out_channels, downsampler = None, stride = 1, batchnorm=False):
        
        super(block, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 2)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = stride)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsampler = downsampler
        self.relu = nn.ReLU()
        self.batchnorm = batchnorm

    
    def forward(self, x):

        residual = x
        x = self.conv1(x)
        if self.batchnorm:
            x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        if self.batchnorm:
            x = self.bn2(x)
        x = self.relu(x)
        
        if self.downsampler:
            residual = self.downsampler(residual)

        return self.relu(residual + x)
    


def ResNet10_Attention(num_classes = 100, batchnorm= False):

    return ResNet_Attention(block, [1, 1, 1, 1], num_classes=num_classes, batchnorm=batchnorm)

In [71]:
## Resnet with Attention

learning_rate = 1e-3

# TODO: Use the above Attention module after the 2nd resnet block i.e. after self.layer2.

model = ResNet10_Attention()

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, optimizer, epochs=10)

Epoch 0, Iteration 0, loss = 4.6222
Checking accuracy on validation set
Got 117 / 1000 correct (11.70)

Epoch 0, Iteration 100, loss = 1.4455
Checking accuracy on validation set
Got 399 / 1000 correct (39.90)

Epoch 0, Iteration 200, loss = 1.4561
Checking accuracy on validation set
Got 452 / 1000 correct (45.20)

Epoch 0, Iteration 300, loss = 1.4625
Checking accuracy on validation set
Got 496 / 1000 correct (49.60)

Epoch 0, Iteration 400, loss = 1.4097
Checking accuracy on validation set
Got 544 / 1000 correct (54.40)

Epoch 0, Iteration 500, loss = 1.5534
Checking accuracy on validation set
Got 546 / 1000 correct (54.60)

Epoch 0, Iteration 600, loss = 1.4604
Checking accuracy on validation set
Got 507 / 1000 correct (50.70)

Epoch 0, Iteration 700, loss = 1.2995
Checking accuracy on validation set
Got 560 / 1000 correct (56.00)

Epoch 1, Iteration 0, loss = 1.1074
Checking accuracy on validation set
Got 532 / 1000 correct (53.20)

Epoch 1, Iteration 100, loss = 1.1402
Checking acc

Epoch 9, Iteration 600, loss = 0.2418
Checking accuracy on validation set
Got 778 / 1000 correct (77.80)

Epoch 9, Iteration 700, loss = 0.3013
Checking accuracy on validation set
Got 780 / 1000 correct (78.00)

Maximum accuracy attained:  79.7


## Test set -- run this only once

Now we test our model on the test set . Think about how this compares to your validation set accuracy.

In [72]:
AttentionResnet = model
check_accuracy(loader_test, AttentionResnet)

Checking accuracy on test set
Got 7656 / 10000 correct (76.56)


76.55999999999999

## Inline Question 3: Rank the above models based on their performance on test dataset (15 points)
( You are encouraged to run each of the experiments (training) at
least 3 times to get an average estimate )

Report the test accuracies alongside the model names. For example, 1. Vanilla CNN (57.45%, 57.99%).. etc

1. <br />Attention ResNet10 (75.8%, 76.98%, 76.56%)
2. <br />Vanilla ResNet10 (74.59%, 74.94%, 74.3%)
3. <br />Double Attention Blocks CNN (61.91%, 62.42%, 63.67%)
4. <br />Single Early Attention CNN (60.5%, 60.91%, 60.08%, 61.26%, 61.43%) - avg: 60.84%
5. <br />Single Late Attention CNN (60.18%, 61.29%, 60.04%, 59.05%, 61.74%) - avg: 60.46%
6. <br />Vanilla CNN (57.25%, 59.4%, 57.98%)

### Bonus Question (Ungraded): Can you give a possible explanation that supports the rankings?
Your Answer: Residual Network provids a skip connection between every two layers while all layers are connected directly. This architecture of residual block allows the network to learn parameters from more deeper layers than classical CNN which may face issues of gradient vanishing when learning from deep layers. Based on that, ResNet10 has overall better performance than the Vanilla CNN has. 
Attention mechanism can lead model to pay "attention" to specific features which increases the accuracy as well. The ranks of single early attention and single late attention are hard to determined since the difference of accuracy between two stategies is relatively low. On the other hand, the double attention blocks indeed have better improvement on accuracy than the single attention block has. 